## Fairness processing

In [20]:
%load_ext autoreload
%autoreload 2
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from aif360.sklearn.metrics import average_odds_difference
from aif360.sklearn.metrics import equal_opportunity_difference
from aif360.sklearn.inprocessing import AdversarialDebiasing

from sklego.linear_model import EqualOpportunityClassifier
from fairlearn.postprocessing import ThresholdOptimizer

from utils import read_diabetes_dataset
from utils import statistical_parity
from utils import average_odds
from utils import average_predictive_value
from utils import theil_index
from utils import disparate_impact

from utils import Weighted
from utils import CounterfactualPreProcessing


import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
BALANCE = False

# protected_atributes = ['race_Caucasian']
# privileged_class = [True]

protected_atributes = ['gender']
privileged_class = [1]

# protected_atributes = ['gender', 'race_Caucasian', 'race_AfricanAmerican', 'race_Asian', 'race_Hispanic', 'race_Other']
# privileged_class = [1, True, False, False, False, False]
# protected_atributes = ['gender', 'race_Caucasian']
# privileged_class = [0, True]



In [22]:
def evaluate_results(y_train, pred_train, Z_train, y_test, pred_test, Z_test):
    
    acc = accuracy_score(y_train, pred_train)
    print('Accuracy', acc)
    bacc = balanced_accuracy_score(y_train, pred_train)
    print('Balanced Accuracy', bacc)
    auc = roc_auc_score(y_train, pred_train)
    print('AUC', auc)
    eqo = equal_opportunity_difference(y_train, y_train_pred, prot_attr=Z_train.squeeze())
    print('Equal opportunity', eqo)
    sp = statistical_parity(y_train, y_train_pred, Z_train.squeeze())
    sp = sp[0]
    print('Statistical parity', sp)
    di = disparate_impact(pred_train, Z_train, privileged_class)
    print('Disparate Impact', di)
    ao =average_odds_difference(y_train, y_train_pred, prot_attr=Z_train.squeeze())
    print('Average odds', ao)
    ti = theil_index(y_train, y_train_pred)
    print('Theil index', ti)

    # print('{:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}'.format(bacc, auc, eqo, sp, di, ao, ti))
    print('{:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f}'.format(bacc, auc, eqo, sp, di, ao, ti))
    print("----------------------")

    acc = accuracy_score(y_test, pred_test)
    print('Accuracy', acc)
    bacc  = balanced_accuracy_score(y_test, pred_test)
    print('Balanced Accuracy', bacc)
    auc =roc_auc_score(y_test, pred_test)
    print('AUC', auc)
    di = disparate_impact(pred_test, Z_test, privileged_class)
    print('Disparate Impact', di)
    sp = statistical_parity(y_test, y_test_pred, Z_test.squeeze())
    sp = sp[0]
    print('Statistical parity', sp)
    ao = average_odds_difference(y_test, y_test_pred, prot_attr=Z_test.squeeze())
    print('Average odds', ao)
    eqo = equal_opportunity_difference(y_test, y_test_pred, prot_attr=Z_test.squeeze())
    print('Equal opportunity', eqo)
    ti = theil_index(y_test, y_test_pred)
    print('Theil index', ti)
    # print('{:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}'.format(bacc, auc, eqo, sp, di, ao, ti))
    print('{:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f}'.format(bacc, auc, eqo, sp, di, ao, ti))



### Dataset

In [23]:
X, y = read_diabetes_dataset(binary=True, use_paper_setup=False)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101)


In [5]:

# column_name = X_train.columns

if BALANCE:
    X_train,y_train = SMOTE().fit_resample(X_train,y_train)


In [6]:
Z_train = (X_train[protected_atributes]).values
Z_test = (X_test[protected_atributes]).values

### Without Mitigation

In [7]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)

lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

Accuracy 0.591246699741782
Balanced Accuracy 0.591246699741782
AUC 0.591246699741782
Equal opportunity 0.452012264037567
[0] Statistical parity -0.4524261392789609
Disparate Impact 3.988495575221239
Average odds 0.43857198642236506
Theil index 0.28885620179645066
0.591 & 0.591 & 0.452 & -0.452 & 3.988 & 0.439 & 0.289
----------------------
Accuracy 0.564667225950783
Balanced Accuracy 0.5582047760108177
AUC 0.5582047760108176
Disparate Impact 3.285055957867018
[0] Statistical parity -0.4292768898090623
Average odds 0.4273940154087035
Equal opportunity 0.4196403975353356
Theil index 0.2712717830621021
0.558 & 0.558 & 0.420 & -0.429 & 3.285 & 0.427 & 0.271


### Pre-processing

- Weighted

In [8]:
prot_atributes_ids = [np.argwhere(X.columns==protected_atributes[i])[0][0] for i in range(len(protected_atributes))]

lr_weighted = Weighted(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=prot_atributes_ids)
lr_weighted.fit(X_train.values, y_train)

y_train_pred = lr_weighted.predict(X_train.values)
y_test_pred = lr_weighted.predict(X_test.values)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

/home/texs/Documentos/IC/TopicosIA/dataset_diabetes/utils.py:350: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  X[:, self.sensitive_feature_ids], y[:, np.newaxis], axis=1)


Accuracy 0.5763048713261961
Balanced Accuracy 0.576304871326196
AUC 0.576304871326196
Equal opportunity 0.021313046454164097
[0] Statistical parity -0.027367812995636065
Disparate Impact 1.4267288493769583
Average odds 0.012027398989496174
Theil index 0.3038606579937587
0.576 & 0.576 & 0.021 & -0.027 & 1.427 & 0.012 & 0.304
----------------------
Accuracy 0.5712388143176734
Balanced Accuracy 0.5662723991414209
AUC 0.5662723991414208
Disparate Impact 1.1530179445350734
[0] Statistical parity -0.007033900959992345
Average odds 0.006087747496101914
Equal opportunity 0.0035469791081130797
Theil index 0.2632084192720996
0.566 & 0.566 & 0.004 & -0.007 & 1.153 & 0.006 & 0.263


- Counterfactual

In [9]:
lr_counter = CounterfactualPreProcessing(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=prot_atributes_ids)
lr_counter.fit(X_train.values, y_train)

y_train_pred = lr_counter.predict(X_train.values)
y_test_pred = lr_counter.predict(X_test.values)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

(137868, 29) (137868,)
Accuracy 0.5798154756723822
Balanced Accuracy 0.5798154756723823
AUC 0.5798154756723823
Equal opportunity 0.0734668321482449
[0] Statistical parity -0.07867168908655742
Disparate Impact 1.587014982712255
Average odds 0.06317033268656497
Theil index 0.2981778557420493
0.580 & 0.580 & 0.073 & -0.079 & 1.587 & 0.063 & 0.298
----------------------
Accuracy 0.5692114093959731
Balanced Accuracy 0.5649027167373019
AUC 0.5649027167373019
Disparate Impact 1.286696459264352
[0] Statistical parity -0.05780274204795177
Average odds 0.05669697325257217
Equal opportunity 0.05326950451402601
Theil index 0.26234258830153234
0.565 & 0.565 & 0.053 & -0.058 & 1.287 & 0.057 & 0.262


### In-processing

- Fairness mitigation

In [10]:
Xn_train = X_train.drop(protected_atributes, axis=1)
Xn_test = X_test.drop(protected_atributes, axis=1)

In [11]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(Xn_train, y_train)

y_train_pred = lr.predict(Xn_train)
y_test_pred = lr.predict(Xn_test)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

Accuracy 0.5796704093770854
Balanced Accuracy 0.5796704093770854
AUC 0.5796704093770854
Equal opportunity 0.07324973611646357
[0] Statistical parity -0.07841899154025056
Disparate Impact 1.5862466384940452
Average odds 0.06294547245923182
Theil index 0.2983666906136223
0.580 & 0.580 & 0.073 & -0.078 & 1.586 & 0.063 & 0.298
----------------------
Accuracy 0.5692813199105146
Balanced Accuracy 0.5649614159624721
AUC 0.5649614159624721
Disparate Impact 1.2871389270976616
[0] Statistical parity -0.057952040345951195
Average odds 0.056933536171288324
Equal opportunity 0.05396401622703939
Theil index 0.2623382897875576
0.565 & 0.565 & 0.054 & -0.058 & 1.287 & 0.057 & 0.262


- Counterfactual appends

In [12]:
X_train_cf =  X_train.copy()
X_train_cf[protected_atributes] = X_train_cf[protected_atributes].replace({0:1, 1:0})
X_train_cf = pd.concat([X_train_cf, X_train])
y_train_cf = pd.concat([y_train, y_train])

X_test_cf =  X_test.copy()
X_test_cf[protected_atributes] = X_test_cf[protected_atributes].replace({0:1, 1:0})
X_test_cf = pd.concat([X_test_cf, X_test])
y_test_cf = pd.concat([y_test, y_test])

In [13]:
# Logistic Regression
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(X_train_cf, y_train_cf)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

Accuracy 0.579684916006615
Balanced Accuracy 0.5796849160066151
AUC 0.5796849160066151
Equal opportunity 0.07393047483428161
[0] Statistical parity -0.07889252252313733
Disparate Impact 1.5877786318216756
Average odds 0.06342711423128161
Theil index 0.2982955424063117
0.580 & 0.580 & 0.074 & -0.079 & 1.588 & 0.063 & 0.298
----------------------
Accuracy 0.5688618568232662
Balanced Accuracy 0.5645537880904812
AUC 0.5645537880904812
Disparate Impact 1.2854689110271385
[0] Statistical parity -0.0573726703007566
Average odds 0.05634559347339402
Equal opportunity 0.05331655750253306
Theil index 0.262529637735382
0.565 & 0.565 & 0.053 & -0.057 & 1.285 & 0.056 & 0.263


### Post-processing

In [14]:
# Z_train = X_train[protected_atributes]
# Z_test = X_test[protected_atributes]

# Z_train = Z_train.values
# Z_test = Z_test.values

In [15]:
# Z_train.shape

- Model

In [16]:
lr = LogisticRegression(class_weight=None, max_iter=10**3)
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

Accuracy 0.591246699741782
Balanced Accuracy 0.591246699741782
AUC 0.591246699741782
Equal opportunity 0.452012264037567
[0] Statistical parity -0.4524261392789609
Disparate Impact 3.988495575221239
Average odds 0.43857198642236506
Theil index 0.28885620179645066
0.591 & 0.591 & 0.452 & -0.452 & 3.988 & 0.439 & 0.289
----------------------
Accuracy 0.564667225950783
Balanced Accuracy 0.5582047760108177
AUC 0.5582047760108176
Disparate Impact 3.285055957867018
[0] Statistical parity -0.4292768898090623
Average odds 0.4273940154087035
Equal opportunity 0.4196403975353356
Theil index 0.2712717830621021
0.558 & 0.558 & 0.420 & -0.429 & 3.285 & 0.427 & 0.271


- Optimizer

In [17]:
postprocess_est = ThresholdOptimizer(
    estimator=lr,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba')
postprocess_est.fit(X_train, y_train, sensitive_features=Z_train)

y_train_pred = postprocess_est.predict(X_train, sensitive_features=Z_train)
y_test_pred = postprocess_est.predict(X_test, sensitive_features=Z_test)


evaluate_results(y_train, y_train_pred, Z_train, y_test, y_test_pred, Z_test)

Accuracy 0.577305828763745
Balanced Accuracy 0.577305828763745
AUC 0.577305828763745
Equal opportunity -0.003438223761880743
[0] Statistical parity -0.0059559652359335025
Disparate Impact 1.363878093679689
Average odds -0.00990569695616722
Theil index 0.2912150050743401
0.577 & 0.577 & -0.003 & -0.006 & 1.364 & -0.010 & 0.291
----------------------
Accuracy 0.5645973154362416
Balanced Accuracy 0.5633567337567902
AUC 0.5633567337567902
Disparate Impact 1.1070991432068544
[0] Statistical parity 0.012229740833386793
Average odds -0.0122122425027274
Equal opportunity -0.009823751748432641
Theil index 0.25565638967711724
0.563 & 0.563 & -0.010 & 0.012 & 1.107 & -0.012 & 0.256


In [18]:
# procesor = AdversarialDebiasing()

# procesor.fit(X_train, y_train)